### Recommendation using reviews - NLP (TF-IDF)

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import dateutil.parser as parser
from datetime import datetime, date, timedelta
import torch
import skorch
import scipy
import torch.nn as nn
import torch
import torch.nn.functional as F
import sys
from skorch.helper import DataFrameTransformer
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error, r2_score
from sklearn import metrics
from sklearn.preprocessing import FunctionTransformer
from skorch.callbacks import EarlyStopping
from sklearn.pipeline import Pipeline
from skorch import NeuralNetRegressor
import pickle
import emoji
import requests
from bs4 import BeautifulSoup
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from nltk.tokenize.treebank import TreebankWordDetokenizer

In [3]:
# replace this with new merged dataset once created
initial_df = pd.read_csv('Data/Merged_TA_Gmaps_Dataset.csv')



In [6]:
df = initial_df.drop(['Unnamed: 0'],axis=1)

# Converting to Lowercase
df["review_full"] = df["review_full"].str.lower()

processed_tweet = re.sub(r'\W', ' ', processed_tweet)


# Removing punctuation and replacing with a space - this is to take care of any events where
# a reviewer has not left a space between their punctuation and words
import string
df['review_full'] = df['review_full'].apply(lambda x: ''.join([' ' if char in string.punctuation else char for char in x]))

# Removing all double spaces that may be created by the above, or occur naturally in the dataset
df['review_full'] = df['review_full'].str.replace('  ', ' ')


# Removing stopwords
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])
df["review_full"] = df["review_full"].apply(lambda text: stopwords(text))

# Removing URLS
def urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

df["review_full"] = df["review_full"].apply(lambda text: urls(text))

# Implementing Lemmatisation to get the base form of each word

# lemmatizer = WordNetLemmatizer()
# def lemming(text):
#     a = word_tokenize(text)
#     answer = list(map(lambda x: lemmatizer.lemmatize(x), a))
#     return answer

# df["review_full"] = df["review_full"].apply(lambda text: lemming(text))

df[['review_full']].sample(5)

ValueError: the first two maketrans arguments must have equal length